In [1]:
!python -m pip install --upgrade pip

In [ ]:
%pip install pandas
%pip install geopy
%pip install pycountry
%pip install datetime

from datetime import datetime
import pandas as pd
import os as os
from geopy.geocoders import Nominatim
from countryinfo import CountryInfo

In [ ]:
percorso_csv = r"../csv"

if os.path.exists(percorso_csv):
    for file in os.listdir(percorso_csv):
        percorso_csv_file = os.path.join(percorso_csv, file)
        print(f"Il file {percorso_csv_file} esiste.")
else:
    print(f"Il file {percorso_csv_file} non è stato trovato.")

In [4]:
df_club_games = pd.read_csv(percorso_csv + "/club_games.csv")
df_clubs = pd.read_csv(percorso_csv + "/clubs.csv")
df_game_events = pd.read_csv(percorso_csv + "/game_events.csv")
df_games = pd.read_csv(percorso_csv + "/games.csv")
df_players = pd.read_csv(percorso_csv + "/players.csv")


C:\Users\gissi\AppData\Local\Temp\ipykernel_11024\1344331387.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_game_lineups = pd.read_csv(percorso_csv + "/game_lineups.csv")


In [6]:
#da player posso eliminare nome e cognome che tanto esiste una colonna che li ha entrambi
elimino = ["first_name", "last_name"]
df_players.drop(columns = elimino, inplace = True)

In [7]:
#nella tabella players ho visto che se c'è position c'è anche la subposition, se invece la position è Missing allora subposition è null --> risolvo mettendo missing
df_players['sub_position'] = df_players['sub_position'].fillna("Missing")

In [9]:
#tabella players dove manca foot mettiamo destro che è il più comune nella popolazione mondiale ed anche nei calciatori
df_players['foot'] = df_players['foot'].fillna("right")

In [ ]:
#Sotituiamo l'altezza mancante con la media di quel ruolo
#Calcola la media dell'altezza per ciascun ruolo
media_altezza_per_ruolo = df_players.groupby('position')['height_in_cm'].mean().round()
 
#Sostituisci i valori mancanti nella colonna 'height_in_cm' con la media del ruolo
df_players['height_in_cm'] = df_players['height_in_cm'].fillna(df_players['position'].map(media_altezza_per_ruolo)

In [13]:
#tabella players per la scadenza del contratto i valori nulli li sostituisco con una data di default
df_players['contract_expiration_date'] = df_players['contract_expiration_date'].fillna("Missing")

In [15]:
#in players per il procuratore invece mettiamo sconosciuto
df_players['agent_name'] = df_players['agent_name'].fillna("Sconosciuto")

In [17]:
#in players dove non c'è valore mercato lo sostituisco col valore minimo di quel ruolo poichè abbiamo visto che sono giocatori minori o ritirati

# Calcola i minimi di 'market_value_in_eur' e 'highest_market_value_in_eur' per ciascun ruolo
min_market_value = df_players.groupby('position')['market_value_in_eur'].min()
min_highest_market_value = df_players.groupby('position')['highest_market_value_in_eur'].min() 
# Sostituisci i valori mancanti nelle colonne 'market_value_in_eur' e 'highest_market_value_in_eur' con i minimi per il ruolo corrispondente
df_players['market_value_in_eur'] = df_players['market_value_in_eur'].fillna(df_players['position'].map(min_market_value)) 
df_players['highest_market_value_in_eur'] = df_players['highest_market_value_in_eur'].fillna(df_players['position'].map(min_highest_market_value))

In [19]:
#in tabella players qui andiamo a riempire i null nei campi cit, country e citizenship con delle funzioni di geopy e counterinfo che ci fanno risalire allo stato/capitale dalla capitale/stato
def da_citta_a_stato(city):
    geolocator = Nominatim(user_agent="google") #user agent can be any user agent
    location = geolocator.geocode(city, language="en", timeout=None) #specified the language as some countries are in other lanaguages
    if location and hasattr(location, "address"):
        country = location.address.split(',')[-1].strip()
        return country
    else:
        return "Sconosciuto"
   
def valori_mancanti(player):
    if pd.isnull(player["country_of_citizenship"]) and pd.isnull(player["country_of_birth"]) and pd.isnull(player["city_of_birth"]):
        df_players.at[index, "country_of_birth"] = "Sconosciuto"
        df_players.at[index,"city_of_birth"] = "Sconosciuto"
        df_players.at[index,"country_of_citizenship"] = "Sconosciuto"
    else:
        if pd.isnull(player["country_of_citizenship"]):
            df_players.at[index,"country_of_citizenship"] = "Sconosciuto"
 
        if pd.isnull(player["country_of_birth"]) and pd.isnull(player["city_of_birth"]):
            df_players.at[index,"country_of_birth"] = "Sconosciuto"
            df_players.at[index,"city_of_birth"] = "Sconosciuto"
        else:
            if pd.isnull(player["country_of_birth"]) and not pd.isnull(player["city_of_birth"]):
                df_players.at[index,"country_of_birth"] = da_citta_a_stato(player["city_of_birth"])
            else:
                if pd.isnull(player["city_of_birth"]) and not pd.isnull(player["country_of_birth"]):
                    df_players.at[index,"city_of_birth"] = CountryInfo(player["country_of_birth"]).capital()
 
for index, player in df_players.iterrows():
    valori_mancanti(player)

In [20]:
#tabella games risolviamo la mancanza della posizione in classifica mettendo la mediana E' DA VALUTARE POICHè MANCA IL 30% DEI DATI
mediana = df_games["home_club_position"].median()
df_games['home_club_position'] = df_games['home_club_position'].fillna(mediana)
df_games['away_club_position'] = df_games['away_club_position'].fillna(mediana)

In [22]:
#in games risolviamo mancanza spettatori mettendo la media di tutte le partite
df_games['attendance'] = df_games['attendance'].fillna(df_games['attendance'].mean().round())

In [24]:
#in games quando arbitro non c'è metto sconosciuto
df_games['referee'] = df_games['referee'].fillna("Sconosciuto")

In [26]:
#elimino le colonne home/away formation perchè manca nel 95% TABELLA: games
cancella = ["home_club_formation", "away_club_formation"]
df_games.drop(columns = cancella, inplace = True)

In [27]:
#se non ci sta il nome dei club su TABELLA: game vuol dire che manca l'id in TABELLA: club (vincolo non rispettato) per cui elimino la riga
df_games = df_games.loc[~df_games[['home_club_name', 'away_club_name']].isnull().any(axis=1)]

In [29]:
#tabella games se manca allenatore metto sconosciuto
df_games['home_club_manager_name'] = df_games['home_club_manager_name'].fillna("Sconosciuto")
df_games['away_club_manager_name'] = df_games['away_club_manager_name'].fillna("Sconosciuto")

In [31]:
#nella tabella GAME_EVENTS inserisco i valori mancanti nel player_in_id
df_game_events['player_in_id'] = df_game_events['player_in_id'].fillna(0)

In [32]:
#nella tabella GAME_EVENTS inserisco i valori mancanti nel player_assist_id
df_game_events['player_assist_id'] = df_game_events['player_assist_id'].fillna(0)
 
#conversione degli id in interi
df_game_events['player_in_id'] = df_game_events['player_in_id'].astype(int)
df_game_events['player_assist_id'] = df_game_events['player_assist_id'].astype(int)
df_game_events["date"] = pd.to_datetime(df_game_events["date"])
df_game_events = df_game_events[df_game_events['game_id'].isin(df_games['game_id'])]

In [36]:
# Ottenere i 'club_id' da df_club_games solo quando 'own_manager_name' o 'opponent_manager_name' ha valori nulli
club_ids_with_null_manager = df_club_games.loc[df_club_games['own_manager_name'].isnull() | df_club_games['opponent_manager_name'].isnull(), 'club_id']
# Verifica se i 'club_id' ottenuti sono presenti in 'df_clubs'
rows_to_drop = df_club_games[~df_club_games['club_id'].isin(df_clubs['club_id']) & club_ids_with_null_manager].index.tolist() 
# Elimina le righe dopo aver ottenuto l'elenco degli indici da eliminare
df_club_games.drop(rows_to_drop, inplace=True)

In [ ]:
#VADO A PRENDERE I NOMI DEGLI ALLENATORI DA GAMES E LI METTO IN CLUB_GAMES PER LE SQUADRE
for index, club_g in df_club_games.iterrows():
    if (pd.isnull(club_g['own_manager_name']) and pd.isnull(club_g['opponent_manager_name'])):
        game_id = club_g['game_id']
        club_id = club_g['club_id']
        manager_info = df_games[df_games['game_id'] == game_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            club_info_2 = df_games[df_games['away_club_id'] == club_id]
            if not club_info.empty:
                df_club_games.at[index, 'own_manager_name'] = manager_info.iloc[0]['home_club_manager_name']
            if not club_info_2.empty:
                df_club_games.at[index, 'opponent_manager_name'] = manager_info.iloc[0]['away_club_manager_name']
            else:
                print("No match found for club_id in home_club_id or away_club_id")
        else:
            df_club_games.drop(index, inplace=True)
    elif (pd.isnull(club_g['own_manager_name']) or pd.isnull(club_g['opponent_manager_name'])):
        game_id = club_g['game_id']
        club_id = club_g['club_id']
        manager_info = df_games[df_games['game_id'] == game_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            if not club_info.empty:
                df_club_games.at[index, 'own_manager_name'] = manager_info.iloc[0]['home_club_manager_name']
            else:
                club_info_2 = df_games[df_games['away_club_id'] == club_id]
                if not club_info_2.empty:
                    df_club_games.at[index, 'opponent_manager_name'] = manager_info.iloc[0]['away_club_manager_name']
                else:
                    print("No match found for club_id in home_club_id or away_club_id")
        else:
            df_club_games.drop(index, inplace=True)

In [ ]:
#VADO A PRENDERE LE POSIZIONI DELLE SQUADRE DA GAMES E LI METTO IN CLUB_GAMES
for index, club_g in df_club_games.iterrows():
    if (pd.isnull(club_g['own_position']) and pd.isnull(club_g['opponent_position'])):
        game_id = club_g['game_id']
        club_id = club_g['club_id']
        manager_info = df_games[df_games['game_id'] == game_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            club_info_2 = df_games[df_games['away_club_id'] == club_id]
            if not club_info.empty:
                df_club_games.at[index, 'own_position'] = manager_info.iloc[0]['home_club_position']
            if not club_info_2.empty:
                df_club_games.at[index, 'opponent_position'] = manager_info.iloc[0]['away_club_position']
            else:
                print("No match found for club_id in home_club_id or away_club_id")
        else:
            df_club_games.drop(index, inplace=True)
    elif (pd.isnull(club_g['own_manager_name']) or pd.isnull(club_g['opponent_manager_name'])):
        game_id = club_g['game_id']
        club_id = club_g['club_id']
        manager_info = df_games[df_games['game_id'] == game_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            if not club_info.empty:
                df_club_games.at[index, 'own_position'] = manager_info.iloc[0]['home_club_position']
            else:
                club_info_2 = df_games[df_games['away_club_id'] == club_id]
                if not club_info_2.empty:
                    df_club_games.at[index, 'opponent_position'] = manager_info.iloc[0]['away_club_position']
                else:
                    print("No match found for club_id in home_club_id or away_club_id")
        else:
            df_club_games.drop(index, inplace=True)

In [ ]:
#INSERISCO I NOMI DEGLI ALLENATORI NELLA TABELLA CLUB PRENDENDOLI DA GAME
for index, club in df_clubs.iterrows():
    if (pd.isnull(club['coach_name'])):
        club_id = club['club_id']
        manager_info = df_games[df_games['home_club_id'] == club_id]
        if not manager_info.empty:
            club_info = df_games[df_games['home_club_id'] == club_id]
            if not club_info.empty:
                df_clubs.at[index, 'coach_name'] = manager_info.iloc[0]['home_club_manager_name']

In [39]:
#dove manca average_age nella tabella CLUB mettiamo la media che è 25.7
df_clubs['average_age'] = df_clubs['average_age'].fillna(25.7) 

In [41]:
#da CLUBS posso eliminare la percentuale e il numero di stranieri e il valore totale della squadra perché inutile
elimino = ["foreigners_number", "foreigners_percentage", "total_market_value"]
df_clubs.drop(columns = elimino, inplace = True)

In [48]:
#creo i csv a fine etl
df_players.to_csv('../csv_post_etl/players_post.csv', index=False)
df_games.to_csv('../csv_post_etl/games_post.csv', index=False)
df_club_games.to_csv('../csv_post_etl/club_games_post.csv', index=False)
df_clubs.to_csv('../csv_post_etl/clubs_post.csv', index=False)
df_game_events.to_csv('../csv_post_etl/game_events_post.csv', index=False)